# Setting up Dev Mode

If you would like to work on the PySyft codebase, you can set up PySyft in dev mode. You will need to clone the repository, install syft locally and run the code you installed

## Cloning the Repo

First, we start by cloning the repo

If you have an SSH key enabled in your github account, use

`git clone git@github.com:OpenMined/PySyft.git`

Otherwise use

`git clone https://github.com/OpenMined/PySyft.git`

## Installing Syft

To install Syft `cd` into the directory in which you cloned PySyft and type

```bash
pip install -e packages/syft
```

This installs `syft` in editable mode, such any change in code are reflected in your environment.

## Running Tox Tests

[Tox](https://tox.wiki/en/latest/) is a project that "aims to automate and standardize testing in Python". For PySyft development, it is used to simplify testing and setting up several environment in a way that works for every developer working on PySyft. You can list the commands that you can execute using `tox-l`, which will give a result similar to this

```
> tox -l

hagrid.publish
lint
stack.test.integration
syft.docs
syft.jupyter
syft.publish
syft.test.security
syft.test.unit
syft.test.notebook
stack.test.notebook
stack.test.integration.enclave.oblv
stack.test.vm
frontend.test.unit
frontend.test.e2e
frontend.generate.types
```

This shows us the list of environments that are specified for PySyft. To see what these environments do, have a look at the `tox.ini` file in the main PySyft repo.

You can run an environment using `tox -e <env_name>`. For instance, to run the unit tests, run

```
tox -e syft.test.unit
```

This tox environment is relatively simple, and just uses pytest to run all the tests for the syft packages. However, some environments are more complicated, and run a series of commands that start multiple processes, docker containers and set up a lot of infrastructure before running the tests. The good thing is that with tox, you dont need to worry about that, you can just run the commands.

## Using Jupyter Environment

Pysyft has a tox command to set up a local jupyter notebook environment, which is useful for development.

```
tox -e syft.jupyter
```

PySyft makes extensive use of jupyter notebook, and a lot of developers use it for experiments when writing code. It can be useful to setup a local gitignore (only for you, not pushed to git) to have a playground where you can experiment, without needing to push files to git, or change the .gitignore. You can do this by adding a folder to your `.git/info/exclude` file, which works similar to the `.gitignore` file, e.g. if we add
```
notebooks/experimental/* 
```
to `.git/info/exclude`, git wont sync the changes to the `experimental` folder to github

`Note:` For developers in MS Windows, before development make sure that your development path does not contain any white spaces in between.

Example:
       
**Invalid Path:** `D:/test space/new env/openmined/PySyft`

**Valid Path:** `D:/test-space/new_env/openmined/PySyft`

The issue with paths containing spaces causing problems on Windows is due to the way that Windows handles file paths, but as long as the development path is free of white spaces, you are good to go. This is not a specific issue related to PySyft.

## Working with Python Domain

PySyft enables a network of computers to connect to each other and do privacy preserving data analysis. The Nodes in the network that hold some data are called `Domains`. When we develop with PySyft, it is very common to start a domain as the first step. `PySyft` makes it very easy to develop against a domain in a notebook by providing an interface (`sy.orchestra`) that allows you to start a domain with a webserver in a notebook in the background, which is a lightweight version of a Domain that would be used in production. You can specify options such as what kind of database you are using, whether you want to use networking and how many processes you want to use. You can launch a Domain by simply executing:

In [1]:
import syft as sy;

In [2]:
node = sy.orchestra.launch(name="dev-mode-example-domain-1", port=8020, reset=True, dev_mode=True);

Starting dev-mode-example-domain-1 server on 0.0.0.0:8020


INFO:     Started server process [2001]


INFO:     Waiting for application startup.


INFO:     Application startup complete.


INFO:     Uvicorn running on http://0.0.0.0:8020 (Press CTRL+C to quit)


INFO:     127.0.0.1:45236 - "GET /api/v2/metadata HTTP/1.1" 200 OK


Waiting for server to start Done.


If we dont need a webserver (for development this is true in many cases), we can omit the port and use. 
```
node = sy.orchestra.launch(name="dev-mode-example-domain-1", dev_mode=True, reset=True)
```

**One of the benefits of not using a port is that you can use a debugger and set breakpoints within api calls. This makes debugging way faster in many cases**

Now, we are ready to start using the domain. The domain comes with standard login credentials for the admin (just for development)

In [3]:
client = node.login(email="info@openmined.org", password="changethis")

Logged into dev-mode-example-domain-1 as <info@openmined.org>


Once you are logged in, you are ready to start using the domain, for instance for creating a dataset (this one is empty, just as a example).

In [4]:
dataset = sy.Dataset(name="my dataset",asset_list=[])
client.upload_dataset(dataset)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

<class 'syft.service.response.SyftSuccess'>: Dataset uploaded to 'dev-mode-example-domain-1'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

Lastly to stop or terminate your Domain, we can execute the following command:

In [5]:
node.land()

Stopping dev-mode-example-domain-1
